In [79]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2

from camera import Camera
import structure
import processor
import features

In [80]:
img1 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0289.JPG')
img2 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0288.JPG')
img3 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0287.JPG')
print("Bilder geladen")

Bilder geladen


In [81]:
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(
    cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), None)
kp2, des2 = sift.detectAndCompute(
    cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), None)
kp3, des3 = sift.detectAndCompute(
    cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY), None)

kp1 = np.array([n.pt for n in kp1])
kp2 = np.array([n.pt for n in kp2])
kp3 = np.array([n.pt for n in kp3])


In [82]:
def matchPoints(kp1,des1,kp2,des2):
    # Find point matches
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=100)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Apply Lowe's SIFT matching ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)

    paare = np.array([[m.queryIdx, m.trainIdx] for m in good])
    
    # Constrain matches to fit homography
    retval, mask = cv2.findHomography(
        kp1[paare[:, 0]], kp2[paare[:, 1]], cv2.RANSAC, 100.0)
    mask = mask.ravel()

    # We select only inlier points
    paare = paare[mask == 1]
    return paare

In [83]:
paare = matchPoints(kp1, des1, kp2, des2)
paare2 = matchPoints(kp2[paare[:, 1]], des2[paare[:, 1]], kp3, des3)


In [84]:
pts1 = kp1[paare[:,0]].T
pts2 = kp2[paare[:,1]].T
pts3 = kp3[paare2[:,1]].T

In [85]:
points1 = processor.cart2hom(pts1)
points2 = processor.cart2hom(pts2)
points3 = processor.cart2hom(pts3)


In [130]:
fig, ax = plt.subplots(1, 3)
ax[0].autoscale_view('tight')
ax[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax[0].plot(points1[0], points1[1], 'r.')
ax[1].autoscale_view('tight')
ax[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[1].plot(points2[0], points2[1], 'r.')
ax[2].autoscale_view('tight')
ax[2].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax[2].plot(kp3[paare2[:, 1], 0], kp3[paare2[:, 1], 1], 'r.')
fig.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [87]:
height, width, ch = img1.shape
intrinsic = np.array([  # for dino
    [3030.65, 0, width / 2-6],
    [0, 3030.65, height / 2+17],
    [0, 0, 1]])
intrinsic

array([[3.03065e+03, 0.00000e+00, 1.99400e+03],
       [0.00000e+00, 3.03065e+03, 1.51700e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00]])

In [88]:
def scale_and_translate_points(points):
    """ Scale and translate image points so that centroid of the points
        are at the origin and avg distance to the origin is equal to sqrt(2).
    :param points: array of homogenous point (3 x n)
    :returns: array of same input shape and its normalization matrix
    """
    x = points[0]
    y = points[1]
    center = points.mean(axis=1)  # mean of each row
    cx = x - center[0]  # center the points
    cy = y - center[1]
    dist = np.sqrt(np.power(cx, 2) + np.power(cy, 2))
    scale = np.sqrt(2) / dist.mean()
    norm3d = np.array([
        [scale, 0, -scale * center[0]],
        [0, scale, -scale * center[1]],
        [0, 0, 1]
    ])

    return np.dot(norm3d, points), norm3d


In [89]:
def correspondence_matrix(p1, p2):
    """Each row in the A matrix below is constructed as
        [x'*x, x'*y, x', y'*x, y'*y, y', x, y, 1]"""
    p1x, p1y = p1[:2]
    p2x, p2y = p2[:2]

    return np.array([
        p1x * p2x, p1x * p2y, p1x,
        p1y * p2x, p1y * p2y, p1y,
        p2x, p2y, np.ones(len(p1x))
    ]).T


In [90]:
def compute_essential_normalized(p1, p2):
    """ Computes the fundamental or essential matrix from corresponding points
        using the normalized 8 point algorithm.
    :input p1, p2: corresponding points with shape 3 x n
    :returns: fundamental or essential matrix with shape 3 x 3
    """
    n = p1.shape[1]
    if p2.shape[1] != n:
        raise ValueError('Number of points do not match.')

    # preprocess image coordinates
    p1n, T1 = scale_and_translate_points(p1)
    p2n, T2 = scale_and_translate_points(p2)

    # compute F or E with the coordinates
    A = correspondence_matrix(p1n, p2n)
    # compute linear least square solution
    U, S, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3)

    # constrain F. Make rank 2 by zeroing out last singular value
    U, S, V = np.linalg.svd(F)
    S[-1] = 0
    S = [1, 1, 0]  # Force rank 2 and equal eigenvalues
    F = U @ np.diag(S) @ V

    # reverse preprocessing of coordinates
    # We know that P1' E P2 = 0
    F = T1.T@F@T2

    return F / F[2, 2]


In [91]:
# Calculate essential matrix with 2d points.
# Result will be up to a scale
# First, normalize points
points1n = np.dot(np.linalg.inv(intrinsic), points1)
points2n = np.dot(np.linalg.inv(intrinsic), points2)
#cv2.undistortPoints(pts1, intrinsic, None)[:,0,:].T

In [92]:
E = compute_essential_normalized(points1n, points2n)
#E1 = structure.compute_essential_normalized(points1n, points2an)
print('Computed essential matrix:', (-E / E[0][1]))


Computed essential matrix: [[-0.03042558 -1.         -0.21811249]
 [ 0.95929048 -0.01430969 -0.00848929]
 [ 0.23792829  0.21658999  0.04594701]]


In [93]:
P1 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
P1

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]])

In [94]:
_,R,t,_= cv2.recoverPose(E,pts1.T,pts2.T, intrinsic)
R = np.linalg.inv(R)
t = -R@t
P2 = np.c_[R,t]
P2


array([[ 0.97900216,  0.04517705, -0.19878081,  0.00546468],
       [-0.04092636,  0.99883817,  0.02544292, -0.21325795],
       [ 0.19969929, -0.0167733 ,  0.97971366,  0.97698065]])

In [95]:
#tripoints3d = structure.reconstruct_points(points1n, points2n, P1, P2)
tripoints3d = structure.linear_triangulation(
    points1n, points2n, P1, P2)


In [96]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'r.')
ax.plot([0], [0], [0], 'g.')
ax.plot(-P2[0, 3], -P2[1, 3], -P2[2, 3], 'g.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
#plt.axis('square')
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [97]:
tripoints3d[:,paare2[:,0]].T

array([[-1.00985079, -0.2711323 ,  1.54914591,  1.        ],
       [-0.97663302, -0.44814942,  1.51551669,  1.        ],
       [-0.96977179, -0.49007449,  1.50927709,  1.        ],
       ...,
       [ 0.86850585, -0.14897178,  1.32013891,  1.        ],
       [ 0.8762597 , -0.04041613,  1.33254353,  1.        ],
       [ 0.86893804, -0.14532363,  1.31963021,  1.        ]])

In [98]:
retval, r2,t2,_ = cv2.solvePnPRansac(tripoints3d[:3,paare2[:,0]].T, kp3[paare2[:,1]], intrinsic, None)
retval, r2,t2

(True,
 array([[0.24431999],
        [0.27178257],
        [0.04742149]]),
 array([[-0.936285  ],
        [-0.20689775],
        [ 0.88055136]]))

In [99]:
R2,_ = cv2.Rodrigues(r2)
P3 = np.c_[R2,t2]
P3

array([[ 0.96237148, -0.01352848,  0.2714003 , -0.936285  ],
       [ 0.0791823 ,  0.96937839, -0.2324558 , -0.20689775],
       [-0.25994481,  0.24519893,  0.93397333,  0.88055136]])

In [100]:
points3n = np.dot(np.linalg.inv(intrinsic), points3)
points3n


array([[-0.58855988, -0.59437814, -0.5953235 , ...,  0.13811134,
         0.14129843,  0.13844267],
       [-0.36082361, -0.43854992, -0.45736602, ..., -0.31761943,
        -0.25845193, -0.31562222],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ]])

In [127]:
tripoints3d2 = structure.linear_triangulation(
    points2n.T[paare2[:, 0]].T, points3n, P2, P3)
tripoints3d2.shape

(4, 613)

In [120]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'r.')
ax.plot(tripoints3d2[0], tripoints3d2[1], tripoints3d2[2], 'b.')
ax.plot([0], [0], [0], 'g.')
ax.plot(-P2[0, 3], -P2[1, 3], -P2[2, 3], 'g.')
ax.plot(-t2[0], -t2[1], -t2[2], 'g.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.axis('square')
ax.set_ylim([-1, 1])
ax.set_xlim([-1, 1])
ax.set_zlim([-1, 2])
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


## Some more points

In [128]:
paare3 = matchPoints(kp2, des2, kp3, des3)


In [116]:
pts2_3 = kp2[paare3[:, 0]].T
pts3_2 = kp3[paare3[:, 1]].T
points2_3 = processor.cart2hom(pts2_3)
points3_2 = processor.cart2hom(pts3_2)
points2_3n = np.dot(np.linalg.inv(intrinsic), points2_3)
points3_2n = np.dot(np.linalg.inv(intrinsic), points3_2)


In [134]:
fig, ax = plt.subplots(1, 3)
ax[0].autoscale_view('tight')
ax[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax[0].plot(points1[0], points1[1], 'r.')
ax[1].autoscale_view('tight')
ax[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[1].plot(points2[0], points2[1], 'r.')
ax[2].autoscale_view('tight')
ax[2].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax[2].plot(kp3[paare2[:, 1], 0], kp3[paare2[:, 1], 1], 'r.')
ax[1].plot(points2_3[0], points2_3[1], 'b.')
ax[2].plot(points3_2[0], points3_2[1], 'b.')
fig.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [126]:
tripoints3d2 = structure.linear_triangulation(
    points2_3n, points3_2n, P2, P3)
tripoints3d2.shape

(4, 1642)

In [131]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'r.')
ax.plot(tripoints3d2[0], tripoints3d2[1], tripoints3d2[2], 'b.')
ax.plot([0], [0], [0], 'g.')
ax.plot(-P2[0, 3], -P2[1, 3], -P2[2, 3], 'g.')
ax.plot(-t2[0], -t2[1], -t2[2], 'g.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.axis('square')
ax.set_ylim([-1, 1])
ax.set_xlim([-1, 1])
ax.set_zlim([-1, 2])
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [136]:
img4 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0286.JPG')
kp4, des4 = sift.detectAndCompute(
    cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY), None)
kp4 = np.array([n.pt for n in kp4])

paare4 = matchPoints(kp3, des3, kp4, des4)

In [137]:

pts3_4 = kp3[paare4[:, 0]].T
pts4_3 = kp4[paare4[:, 1]].T
points3_4 = processor.cart2hom(pts3_4)
points4_3 = processor.cart2hom(pts4_3)
points3_4n = np.dot(np.linalg.inv(intrinsic), points3_4)
points4_3n = np.dot(np.linalg.inv(intrinsic), points4_3)


In [138]:
fig, ax = plt.subplots(1, 2)
ax[0].autoscale_view('tight')
ax[0].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax[0].plot(points3_4[0], points3_4[1], 'r.')
ax[1].autoscale_view('tight')
ax[1].imshow(cv2.cvtColor(img4, cv2.COLOR_BGR2RGB))
ax[1].plot(points4_3[0], points4_3[1], 'r.')
fig.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
